In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


prices_1 = pd.read_csv('../data/round-4-island-data-bottle/prices_round_4_day_1.csv', sep=';')
prices_2 = pd.read_csv('../data/round-4-island-data-bottle/prices_round_4_day_2.csv', sep=';')
prices_3 = pd.read_csv('../data/round-4-island-data-bottle/prices_round_4_day_3.csv', sep=';')

df_prices = pd.concat([prices_1, prices_2, prices_3]).reset_index(drop=True)
df_prices['timestamp'] = df_prices['timestamp'] + (df_prices['day'] - 1) * 1000000
df_prices = df_prices.drop(columns=['day', 'profit_and_loss'])
df_coup_prices = df_prices[df_prices['product'] == 'COCONUT_COUPON'].reset_index(drop=True).copy()
df_nut_prices = df_prices[df_prices['product'] == 'COCONUT'].reset_index(drop=True).copy()
premium = 637.63
df_coup_prices['mid_price'] = df_coup_prices['mid_price'] - premium
df_nut_prices['mid_price'] = df_nut_prices['mid_price'] - 10000

In [ ]:
df_coup_prices

In [ ]:
# plot both prices over time
plt.figure(figsize=(10, 6))
plt.plot(df_coup_prices['timestamp'], df_coup_prices['mid_price'], label='Coconut Coupon')
plt.plot(df_nut_prices['timestamp'], df_nut_prices['mid_price'], label='Coconut')
plt.xlabel('Timestamp')
plt.ylabel('Price')
plt.legend()
plt.show()

In [ ]:
#plot difference between coupon and nut price over time
df_diff = df_nut_prices.copy()
df_diff['diff'] = df_nut_prices['mid_price'] - df_coup_prices['mid_price']
plt.figure(figsize=(10, 6))
plt.plot(df_diff['timestamp'], df_diff['diff'])
plt.xlabel('Time')
plt.ylabel('Nut - Price')
plt.show()


In [ ]:
#plot changes in the difference over time
df_diff['diff_change'] = df_diff['diff'].diff()
plt.figure(figsize=(10, 6))
plt.plot(df_diff['timestamp'], df_diff['diff_change'])
plt.xlabel('Time')
plt.ylabel('Change in Nut - Price')
plt.show()


#plot a histogram of the changes and a normal distrithe same mean and stdbution with 
plt.figure(figsize=(10, 6))
plt.hist(df_diff['diff_change'], bins=100, density=True)
mean = df_diff['diff_change'].mean()
std = df_diff['diff_change'].std()
x = np.linspace(-5, 5, 100)
y = 1/(std * np.sqrt(2 * np.pi)) * np.exp(- (x - mean)**2 / (2 * std**2))
plt.plot(x, y, 'r')
plt.plot(x, y, 'r')
plt.xlabel('Change in diff t+1 - diff t')
plt.ylabel('Frequency')
plt.show()

# test for normality
from scipy.stats import shapiro
stat, p = shapiro(df_diff['diff_change'].dropna())
print('Statistics=%.3f, p=%.3f' % (stat, p))
#issue: since prices are integers, the changes are integers as well, so the normality test is not valid
# They might still be normally distributed with rounding, but we cannot test it with this test

In [ ]:
#plot a scatter of the difference in price and the price of the nut
plt.figure(figsize=(10, 6))
plt.title('Scatter of difference in price and price of nut')
plt.scatter(df_diff['mid_price'], df_diff['diff'], s=0.1)
plt.show()

#plot scatter of difference in difference and price of nut
plt.figure(figsize=(10, 6))
plt.title('Scatter of difference in difference and price of nut')
plt.scatter(df_diff['mid_price'], df_diff['diff_change'], s=0.1)
plt.show()


In [ ]:
from statsmodels.tsa.stattools import adfuller
#check if data is stationary
result = adfuller(df_diff['diff'].dropna())
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])

#check if differenced data is stationary
result = adfuller(df_diff['diff_change'].dropna())
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))
# difference itself is not stationary but differenced data is stationary 

In [ ]:
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
#plot acf and pacf
plt.figure(figsize=(10, 6))
plot_acf(df_diff['diff_change'].dropna(), lags=10, zero=False)
plt.show()

plt.figure(figsize=(10, 6))
plot_pacf(df_diff['diff_change'].dropna(), lags=10, zero=False)
plt.show()

In [ ]:
# fit ARIMA model with d = 1, p = 1, q = 1
model = ARIMA(df_diff['diff'].dropna(), order=(1, 1, 1))
model_fit = model.fit()
print(model_fit.summary())


In [ ]:
#plot residuals of model
plt.figure(figsize=(10, 6))
plt.plot(model_fit.resid)
plt.show()


In [ ]:
#plot model predictions against actual data
plt.figure(figsize=(100, 6))
plt.plot(df_diff['timestamp'][0:1000], df_diff['diff'][0:1000], label='Actual')
plt.plot(df_diff['timestamp'][0:1000], model_fit.fittedvalues[0:1000], label='Predicted')
plt.legend()
plt.show()

# The model seems to only be able to predict a tiny amount with the rest just being noise. It is not a good model.

In [ ]:
#test if diff was already white noise using Ljung-Box test
from statsmodels.stats.diagnostic import acorr_ljungbox
result = acorr_ljungbox(df_diff['diff_change'].dropna(), lags=10)
print(result)
# Looks like the data was not white noise

#test if residuals of arima model are white noise
result = acorr_ljungbox(model_fit.resid, lags=10)
print(result)
#looks like the residuals are white noise

In [ ]:
#other idea: plot abs deviation of cocnut price against difference in price
plt.figure(figsize=(10, 6))
plt.scatter(np.abs(df_diff['mid_price']), np.abs(df_diff['diff']), s=0.1)
#plot regression line
from sklearn.linear_model import LinearRegression
X = np.abs(df_diff['mid_price']).values.reshape(-1, 1)
y = np.abs(df_diff['diff']).values
reg = LinearRegression().fit(X, y)
plt.plot(X, reg.predict(X), 'r')
plt.xlabel('Absolute price of coconut')
plt.ylabel('Difference in price')
plt.show()
# Trading idea: -> if the price of the coconut is high, the difference in price is also high. We can model this
# and use it to predict the difference in price. 

#Need to check whether linear regression is predictive using a few past diffs/price values and trade on that

In [ ]:
from scipy.optimize import brentq
from scipy.stats import norm

# Black scholes
def black_scholes(S, K, T, r, sigma, option_type='call'):
    d1 = (np.log(S/K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if option_type == 'call':
        return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:
        return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    

def get_underlying_price_from_bs(option_price, K, t, r, sigma=0.19226514699995814, option_type='call'):
    def f(S):
        return black_scholes(S, K, t, r, sigma, option_type) - option_price
    return brentq(f, 0.01, 1000)

In [ ]:

def binary_search_vol(market_price, S, K, T, r, tolerance=1e-5):
    sigma_low = 0.001
    sigma_high = 1.0
    for i in range(100):
        sigma_mid = (sigma_low + sigma_high) / 2
        price_mid = black_scholes(S, K, T, r, sigma_mid)
        if price_mid > market_price:
            sigma_high = sigma_mid
        else:
            sigma_low = sigma_mid
            
        if abs(price_mid - market_price) < tolerance:
            return sigma_mid
    
    return (sigma_low + sigma_high) / 2

In [ ]:
# Calculate implied volatility
from scipy.stats import norm
# join coconut and coupon prices
coup_prices = df_coup_prices['mid_price'].values + premium
coconut_prices = df_nut_prices['mid_price'].values + 10000
K = 10000
T = 250 / 365
r = 0
implied_vol = []
for i in range(len(coup_prices)):
    price = coup_prices[i]
    S = coconut_prices[i]
    sigma = binary_search_vol(price, S, K, T, r)
    implied_vol.append(sigma)
print(implied_vol)
print(max(implied_vol))
print(min(implied_vol))

In [ ]:
# get black scholes price for each row
bs_prices = []
for i in range(len(coup_prices)):
    S = coconut_prices[i]
    K = 10000
    T = 250 / 365
    r = 0
    sigma = implied_vol[i]
    price = black_scholes(S, K, T, r, sigma)
    bs_prices.append(price)

In [ ]:
# plot implied volatility over time. Seems mean reverting. 
# Strategy idea: just use mean vol to price the options and trade on that
plt.figure(figsize=(10, 6))
plt.plot(df_coup_prices['timestamp'], implied_vol)
plt.xlabel('Timestamp')
plt.ylabel('Implied Volatility')
plt.show()

In [ ]:
# sanity check: black scholes prices are basically same as the actual prices
print(min(coup_prices - bs_prices))
print(max(coup_prices - bs_prices))

In [ ]:
np.mean(implied_vol)